In [1]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import UpSampling1D
# from keras.layers import Dropout
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D
from keras.models import Model
from keras.utils import plot_model
from sklearn.metrics import mean_squared_error
np.random.seed(7)

Using TensorFlow backend.


In [2]:
# hyperparameters
window_size = 500
stride = 6
# threshold = 0.1

In [3]:
def create_dataset(dataset):
    N = int(dataset.shape[0] / stride)
    X = []
    for i in range(N - stride + 1):
        sample = dataset[i : (i + stride)]
        X.append(sample)
    return(np.array(X))

def array_mse(X, Y):
    mse = []
    for i in range(X.shape[0]):
        mse.append(mean_squared_error(X[i], Y[i]))
    return mse

In [4]:
# device data processing
df = pd.read_csv("data/10.42.0.95.csv", header=None)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,490,491,492,493,494,495,496,497,498,499
0,0.270588,0.0,0.0,0.784314,0.329412,0.937255,0.0,0.0,1.0,0.066667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.270588,0.0,0.0,0.784314,0.329412,0.941176,0.0,0.0,1.0,0.066667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.270588,0.0,0.0,0.784314,0.329412,0.941176,0.0,0.0,1.0,0.066667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.270588,0.0,0.0,0.784314,0.329412,0.945098,0.0,0.0,1.0,0.066667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.270588,0.0,0.0,0.784314,0.329412,0.945098,0.0,0.0,1.0,0.066667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
dataset_train = df.values
X_train = create_dataset(dataset_train)
X_train.shape

(1431, 6, 500)

In [7]:
# define model
model = Sequential()
model.add(LSTM(64, activation='relu', input_shape=(stride, 500)))
model.add(RepeatVector(stride))
model.add(LSTM(64, activation='relu', return_sequences=True))
model.add(TimeDistributed(Dense(500)))
model.compile(optimizer='adam', loss='mse')
model.summary()

W1016 14:22:49.865125 4505378112 deprecation_wrapper.py:119] From /Users/lee/venv3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1016 14:22:49.895430 4505378112 deprecation_wrapper.py:119] From /Users/lee/venv3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1016 14:22:49.908448 4505378112 deprecation_wrapper.py:119] From /Users/lee/venv3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1016 14:22:50.453988 4505378112 deprecation_wrapper.py:119] From /Users/lee/venv3/lib/python3.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 64)                144640    
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 6, 64)             0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 6, 64)             33024     
_________________________________________________________________
time_distributed_1 (TimeDist (None, 6, 500)            32500     
Total params: 210,164
Trainable params: 210,164
Non-trainable params: 0
_________________________________________________________________


In [8]:
# fit model
model.fit(X_train, X_train, epochs=100, verbose=True)

W1016 14:22:57.369021 4505378112 deprecation.py:323] From /Users/lee/venv3/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W1016 14:22:58.728836 4505378112 deprecation_wrapper.py:119] From /Users/lee/venv3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.

W1016 14:22:58.857353 4505378112 deprecation_wrapper.py:119] From /Users/lee/venv3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:973: The name tf.assign is deprecated. Please use tf.compat.v1.assign instead.



Epoch 1/100
1431/1431 [==============================] - 2s 2ms/step - loss: 0.0151
Epoch 2/100
1431/1431 [==============================] - 1s 489us/step - loss: 0.0022
Epoch 3/100
1431/1431 [==============================] - 1s 491us/step - loss: 0.0021
Epoch 4/100
1431/1431 [==============================] - 1s 533us/step - loss: 0.0021
Epoch 5/100
1431/1431 [==============================] - 1s 490us/step - loss: 0.0020
Epoch 6/100
1431/1431 [==============================] - 1s 560us/step - loss: 0.0019
Epoch 7/100
1431/1431 [==============================] - 1s 540us/step - loss: 0.0017
Epoch 8/100
1431/1431 [==============================] - 1s 636us/step - loss: 0.0016
Epoch 9/100
1431/1431 [==============================] - 1s 582us/step - loss: 0.0015
Epoch 10/100
1431/1431 [==============================] - 1s 528us/step - loss: 0.0015
Epoch 11/100
1431/1431 [==============================] - 1s 524us/step - loss: 0.0014
Epoch 12/100
1431/1431 [==============================

1431/1431 [==============================] - 1s 600us/step - loss: 3.2656e-05
Epoch 93/100
1431/1431 [==============================] - 1s 552us/step - loss: 3.2301e-05
Epoch 94/100
1431/1431 [==============================] - 1s 471us/step - loss: 3.4185e-05
Epoch 95/100
1431/1431 [==============================] - 1s 452us/step - loss: 3.3644e-05
Epoch 96/100
1431/1431 [==============================] - 1s 460us/step - loss: 3.3898e-05
Epoch 97/100
1431/1431 [==============================] - 1s 532us/step - loss: 3.1498e-05
Epoch 98/100
1431/1431 [==============================] - 1s 535us/step - loss: 3.7047e-05
Epoch 99/100
1431/1431 [==============================] - 1s 567us/step - loss: 3.0163e-05
Epoch 100/100
1431/1431 [==============================] - 1s 473us/step - loss: 2.9824e-05


In [9]:
# attack data processing
df = pd.read_csv("data/10.37.130.4.csv", header=None)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,490,491,492,493,494,495,496,497,498,499
0,0.270588,0.0,0.0,0.952941,0.003922,0.701961,0.00000,0.0,0.501961,0.066667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.270588,0.0,0.0,0.898039,0.003922,0.713725,0.00000,0.0,0.501961,0.066667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.270588,0.0,0.0,0.952941,0.003922,0.725490,0.00000,0.0,0.501961,0.066667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.270588,0.0,0.0,0.866667,0.003922,0.835294,0.25098,0.0,0.501961,0.023529,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.270588,0.0,0.0,0.674510,0.003922,0.843137,0.25098,0.0,0.501961,0.023529,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
dataset_test = df.values
X_test = create_dataset(dataset_test)
X_test.shape

(8, 6, 500)

In [14]:
# predict
Y = model.predict(X_test, verbose=True)
mse = array_mse(X_test, Y)
mse

8/8 [==============================] - 0s 1ms/step


[0.05490794874512366,
 0.05411165718499836,
 0.059065045483775364,
 0.05774797059245123,
 0.06007812010396116,
 0.0767395394422584,
 0.07303675778139558,
 0.07835029189428072]

In [16]:
X = dataset_train[25: 31]
X = np.array(X)
X = X.reshape((1, stride, 500))
Y = model.predict(X_test, verbose=False)
mse = array_mse(X, Y)
mse

[0.012297331182534553]

In [34]:
model2 = Sequential()
model2.add(LSTM(128, activation='relu', return_sequences=True))
model2.add(LSTM(64, activation='relu', return_sequences=True))
model2.add(LSTM(64, activation='relu', return_sequences=True))
model2.add(LSTM(128, activation='relu', return_sequences=True))
model2.add(TimeDistributed(Dense(500)))
model2.compile(optimizer='adam', loss='mse')
# model2.summary()

In [44]:
model2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_49 (LSTM)               (None, 6, 128)            322048    
_________________________________________________________________
lstm_50 (LSTM)               (None, 6, 64)             49408     
_________________________________________________________________
lstm_51 (LSTM)               (None, 6, 64)             33024     
_________________________________________________________________
lstm_52 (LSTM)               (None, 6, 128)            98816     
_________________________________________________________________
time_distributed_14 (TimeDis (None, 6, 500)            64500     
Total params: 567,796
Trainable params: 567,796
Non-trainable params: 0
_________________________________________________________________


In [35]:
model2.fit(X_train, X_train, epochs=100, verbose=True)

Epoch 1/100
1431/1431 [==============================] - 5s 4ms/step - loss: 0.0268
Epoch 2/100
1431/1431 [==============================] - 1s 1ms/step - loss: 0.0043
Epoch 3/100
1431/1431 [==============================] - 2s 1ms/step - loss: 0.0017
Epoch 4/100
1431/1431 [==============================] - 2s 1ms/step - loss: 7.9507e-04
Epoch 5/100
1431/1431 [==============================] - 2s 1ms/step - loss: 4.6253e-04
Epoch 6/100
1431/1431 [==============================] - 2s 1ms/step - loss: 4.3571e-04
Epoch 7/100
1431/1431 [==============================] - 2s 1ms/step - loss: 4.2751e-04
Epoch 8/100
1431/1431 [==============================] - 2s 1ms/step - loss: 4.2248e-04
Epoch 9/100
1431/1431 [==============================] - 2s 1ms/step - loss: 4.1513e-04
Epoch 10/100
1431/1431 [==============================] - 2s 1ms/step - loss: 4.0680e-04
Epoch 11/100
1431/1431 [==============================] - 2s 1ms/step - loss: 4.0120e-04
Epoch 12/100
1431/1431 [==================

In [36]:
# predict
Y = model2.predict(X_test, verbose=True)
mse = array_mse(X_test, Y)
mse

8/8 [==============================] - 1s 85ms/step


[0.041234305364196,
 0.039579794010896625,
 0.04299033896398769,
 0.0460908496265392,
 0.04579129958554307,
 0.05405099782129351,
 0.05326157375131788,
 0.0628076696276005]

In [37]:
X = dataset_train[25: 31]
X = np.array(X)
X = X.reshape((1, stride, 500))
Y = model2.predict(X_test, verbose=False)
mse = array_mse(X, Y)
mse

[0.006788386238215036]

In [6]:
model3 = Sequential()
model3.add(Conv1D(filters=128, kernel_size=3, activation='relu', padding='same', input_shape=(X_train.shape[1], X_train.shape[2])))
model3.add(MaxPooling1D(pool_size=2, padding='same'))
model3.add(LSTM(64, activation='relu'))
model3.add(RepeatVector(3))
model3.add(LSTM(64, activation='relu', return_sequences=True))
# model3.add(Conv1D(filters=128, kernel_size=3, activation='relu', padding='same'))
model3.add(UpSampling1D(2))
model3.add(TimeDistributed(Dense(500)))
model3.compile(optimizer='adam', loss='mse')
model3.summary()

W1216 10:59:31.277292 4376161600 deprecation_wrapper.py:119] From /Users/lee/venv3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1216 10:59:31.311142 4376161600 deprecation_wrapper.py:119] From /Users/lee/venv3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1216 10:59:31.316984 4376161600 deprecation_wrapper.py:119] From /Users/lee/venv3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1216 10:59:31.352447 4376161600 deprecation_wrapper.py:119] From /Users/lee/venv3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W1216 10:59:31.863112 4376161600 deprecation_wrappe

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 6, 128)            192128    
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 3, 128)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                49408     
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 3, 64)             0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 3, 64)             33024     
_________________________________________________________________
up_sampling1d_1 (UpSampling1 (None, 6, 64)             0         
_________________________________________________________________
time_distributed_1 (TimeDist (None, 6, 500)            32500     
Total para

In [7]:
model3.fit(X_train, X_train, epochs=100, verbose=True)

W1216 10:59:43.650417 4376161600 deprecation.py:323] From /Users/lee/venv3/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W1216 10:59:44.987773 4376161600 deprecation_wrapper.py:119] From /Users/lee/venv3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.

W1216 10:59:45.196118 4376161600 deprecation_wrapper.py:119] From /Users/lee/venv3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:973: The name tf.assign is deprecated. Please use tf.compat.v1.assign instead.



Epoch 1/100
1431/1431 [==============================] - 3s 2ms/step - loss: 0.0164
Epoch 2/100
1431/1431 [==============================] - 1s 517us/step - loss: 0.0023
Epoch 3/100
1431/1431 [==============================] - 1s 509us/step - loss: 0.0021
Epoch 4/100
1431/1431 [==============================] - 1s 497us/step - loss: 0.0021
Epoch 5/100
1431/1431 [==============================] - 1s 550us/step - loss: 0.0020
Epoch 6/100
1431/1431 [==============================] - 1s 515us/step - loss: 0.0019
Epoch 7/100
1431/1431 [==============================] - 1s 519us/step - loss: 0.0019
Epoch 8/100
1431/1431 [==============================] - 1s 592us/step - loss: 0.0018
Epoch 9/100
1431/1431 [==============================] - 1s 503us/step - loss: 0.0017
Epoch 10/100
1431/1431 [==============================] - 1s 506us/step - loss: 0.0016
Epoch 11/100
1431/1431 [==============================] - 1s 541us/step - loss: 0.0015
Epoch 12/100
1431/1431 [==============================

1431/1431 [==============================] - 1s 564us/step - loss: 0.0011
Epoch 96/100
1431/1431 [==============================] - 1s 512us/step - loss: 0.0011
Epoch 97/100
1431/1431 [==============================] - 1s 581us/step - loss: 0.0011
Epoch 98/100
1431/1431 [==============================] - 1s 553us/step - loss: 0.0011
Epoch 99/100
1431/1431 [==============================] - 1s 632us/step - loss: 0.0011
Epoch 100/100
1431/1431 [==============================] - 1s 635us/step - loss: 0.0011


In [61]:
# predict
Y = model3.predict(X_test, verbose=True)
mse = array_mse(X_test, Y)
mse

8/8 [==============================] - 1s 131ms/step


[0.041917398170706895,
 0.04122794614020026,
 0.04670749251025612,
 0.04697457914837163,
 0.04741192861608962,
 0.05383986200815402,
 0.05400260338721894,
 0.064408862900394]

In [62]:
X = dataset_train[25: 31]
X = np.array(X)
X = X.reshape((1, stride, 500))
Y = model3.predict(X_test, verbose=False)
mse = array_mse(X, Y)
mse

[0.005190094391769208]